#### Required python libraries

In [26]:
# pip install pandas numpy matplotlib seaborn

#### Defining the imports

In [27]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
%matplotlib inline    
import matplotlib as mpl
import seaborn as sns
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

#### Loading the provided datasets

* Features Dataset : features.csv :- Additional data related to the store, department, and regional activity for the given dates.
* Train Dataset : train.csv :- This is the historical training data, which covers 2010–02–05 to 2012-08-17.
* Test Dataset : test.csv :- Similar to train.csv, without the weekly sales column
* Stores Dataset : stores.csv :- Contains anonymized information about the 45 stores, indicating the type and size of the store.

In [28]:
df_features = pd.read_csv("dataset/features.csv")
df_store = pd.read_csv("dataset/stores.csv")
df_train = pd.read_csv("dataset/train.csv")
df_test = pd.read_csv("dataset/train.csv")

In [29]:
# Replacing Column Name spaces with "_"
df_features.columns = df_features.columns.str.replace(' ', '_')
df_store.columns = df_store.columns.str.replace(' ', '_')
df_train.columns = df_train.columns.str.replace(' ', '_')

In [30]:
# Changing type of datetime object to datetime and sorting
df_features["Date"] = pd.to_datetime(df_features["Date"])
df_train["Date"] = pd.to_datetime(df_features["Date"])
# Sorting by Date
df_features = df_features.sort_values('Date')
df_train = df_train.sort_values('Date')

#### Exploration of Features Dataset

In [31]:
# Shape of the features data records
df_features.shape

(8190, 12)

In [32]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         8190 non-null   int64         
 1   Date          8190 non-null   datetime64[ns]
 2   Temperature   8190 non-null   float64       
 3   Fuel_Price    8190 non-null   float64       
 4   MarkDown1     4032 non-null   float64       
 5   MarkDown2     2921 non-null   float64       
 6   MarkDown3     3613 non-null   float64       
 7   MarkDown4     3464 non-null   float64       
 8   MarkDown5     4050 non-null   float64       
 9   CPI           7605 non-null   float64       
 10  Unemployment  7605 non-null   float64       
 11  IsHoliday     8190 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(9), int64(1)
memory usage: 775.8 KB


In [33]:
df_features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
2730,16,2010-02-05,19.79,2.580,NaN,NaN,NaN,NaN,NaN,189.381697,7.039,False
5460,31,2010-02-05,39.05,2.572,NaN,NaN,NaN,NaN,NaN,210.752605,8.324,False
3640,21,2010-02-05,39.05,2.572,NaN,NaN,NaN,NaN,NaN,210.752605,8.324,False
4550,26,2010-02-05,9.55,2.788,NaN,NaN,NaN,NaN,NaN,131.527903,8.488,False


In [34]:
df_features.tail()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
5823,32,2013-07-26,72.99,3.582,549.89,940.93,86.00,106.47,1530.56,NaN,NaN,False
2365,13,2013-07-26,83.62,3.669,346.31,1377.41,93.40,140.32,2147.06,NaN,NaN,False
5641,31,2013-07-26,85.00,3.620,1394.82,138.71,12.00,970.77,6859.07,NaN,NaN,False
6733,37,2013-07-26,83.28,3.620,178.00,11.86,NaN,NaN,779.32,NaN,NaN,False
8189,45,2013-07-26,76.06,3.804,212.02,851.73,2.06,10.88,1864.57,NaN,NaN,False


In [52]:
# Columns in the dataset
df_features.columns

Index(['Store', 'Date', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment',
       'IsHoliday'],
      dtype='object')

* Store: The Store Number
* Date: Dates of the data from 2010 February 05th  - 2013 July 26th
* Temperature: Temperature in Fahrenheit
* Fuel Price: Price of Fuel that day
* Markdown1, Markdown2, Markdown3, Markdown4: Data related to promotional markdowns at Walmart which are anonymous
* CPI: The consumer price index which measures the monthly changes in prices paid by US consumers
* Unemployement: Rate of Unemployment
* IsHoliday: Whether a particular day is a holiday

In [35]:
# Summary Statistics
df_features.describe()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,8190.000000,8190,8190.000000,8190.000000,4032.000000,2921.000000,3613.000000,3464.000000,4050.000000,7605.000000,7605.000000
mean,23.000000,2011-10-31 12:00:00,59.356198,3.405992,7032.371786,3384.176594,1760.100180,3292.935886,4132.216422,172.460809,7.826821
min,1.000000,2010-02-05 00:00:00,-7.290000,2.472000,-2781.450000,-265.760000,-179.260000,0.220000,-185.170000,126.064000,3.684000
25%,12.000000,2010-12-17 00:00:00,45.902500,3.041000,1577.532500,68.880000,6.600000,304.687500,1440.827500,132.364839,6.634000
50%,23.000000,2011-10-31 12:00:00,60.710000,3.513000,4743.580000,364.570000,36.260000,1176.425000,2727.135000,182.764003,7.806000
75%,34.000000,2012-09-14 00:00:00,73.880000,3.743000,8923.310000,2153.350000,163.150000,3310.007500,4832.555000,213.932412,8.567000
max,45.000000,2013-07-26 00:00:00,101.950000,4.468000,103184.980000,104519.540000,149483.310000,67474.850000,771448.100000,228.976456,14.313000
std,12.987966,NaN,18.678607,0.431337,9262.747448,8793.583016,11276.462208,6792.329861,13086.690278,39.738346,1.877259


In [36]:
# Null Value Count in the features dataset
df_features.isnull().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

In [37]:
# Unique Values in the features dataset
df_features.nunique()

Store             45
Date             182
Temperature     4178
Fuel_Price      1011
MarkDown1       4023
MarkDown2       2715
MarkDown3       2885
MarkDown4       3405
MarkDown5       4045
CPI             2505
Unemployment     404
IsHoliday          2
dtype: int64

#### Exploration of Store Dataset

In [38]:
df_store.shape

(45, 3)

In [39]:
df_store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     int64 
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


In [40]:
df_store.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [41]:
df_store.tail()

,Store,Type,Size
40,41,A,196321
41,42,C,39690
42,43,C,41062
43,44,C,39910
44,45,B,118221


In [42]:
# Summary Statistics
df_store.describe()

,Store,Size
count,45.000000,45.000000
mean,23.000000,130287.600000
std,13.133926,63825.271991
min,1.000000,34875.000000
25%,12.000000,70713.000000
50%,23.000000,126512.000000
75%,34.000000,202307.000000
max,45.000000,219622.000000


In [43]:
# Null Value Count in the store dataset
df_store.isnull().sum()

Store    0
Type     0
Size     0
dtype: int64

In [44]:
# Unique Values in the store dataset
df_store.nunique()

Store    45
Type      3
Size     40
dtype: int64

#### Exploration of Train Dataset

In [51]:
df_train.shape

(421570, 5)

In [45]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         421570 non-null  int64         
 1   Dept          421570 non-null  int64         
 2   Date          8190 non-null    datetime64[ns]
 3   Weekly_Sales  421570 non-null  float64       
 4   IsHoliday     421570 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(2)
memory usage: 16.5 MB


In [46]:
df_train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
2730,1,21,2010-02-05,8844.39,False
5460,1,41,2010-02-05,663.00,False
3640,1,27,2010-02-05,1752.13,False
4550,1,34,2010-02-05,12571.50,False


In [47]:
df_train.tail()

,Store,Dept,Date,Weekly_Sales,IsHoliday
421565,45,98,NaT,508.37,False
421566,45,98,NaT,628.10,False
421567,45,98,NaT,1061.02,False
421568,45,98,NaT,760.01,False
421569,45,98,NaT,1076.80,False


In [48]:
# Summary Statistics
df_train.describe()

,Store,Dept,Date,Weekly_Sales
count,421570.000000,421570.000000,8190,421570.000000
mean,22.200546,44.260317,2011-10-31 12:00:00,15981.258123
min,1.000000,1.000000,2010-02-05 00:00:00,-4988.940000
25%,11.000000,18.000000,2010-12-17 00:00:00,2079.650000
50%,22.000000,37.000000,2011-10-31 12:00:00,7612.030000
75%,33.000000,74.000000,2012-09-14 00:00:00,20205.852500
max,45.000000,99.000000,2013-07-26 00:00:00,693099.360000
std,12.785297,30.492054,NaN,22711.183519


In [49]:
# Null Value Count in the features dataset
df_train.isnull().sum()

Store                0
Dept                 0
Date            413380
Weekly_Sales         0
IsHoliday            0
dtype: int64

In [50]:
# Unique Values in the features dataset
df_train.nunique()

Store               45
Dept                81
Date               182
Weekly_Sales    359464
IsHoliday            2
dtype: int64